# Init stuffs

### Imports

In [1]:
from collections import deque

### Input data

In [2]:
filename = "./input_1.txt"
test_filename = "./input_2.txt"

with open(file=filename) as f:
    data = f.read().splitlines()

with open(file=test_filename) as f:
    test_data = f.read().splitlines()

# Pt. 1

In [3]:
def init_modules_inputs_outputs_state(module_config: list[str]) -> dict[dict]:
    config_dict = {}
    for line in module_config:
        line_content = line.split()
        if line_content[0][0] == "b":
            dict_index = line_content[0]
        else:
            dict_index = line_content[0][1:]

        config_dict[dict_index] = {"module_type": line_content[0][0],
                                   "module_state": "off" if line_content[0][0] == "%" else [],
                                   "output": "low",
                                   "sources": [], 
                                   "sinks": [x.strip(",") for x in line_content[2:]]}
    # Link all sources
    added_entry = False
    for k, v in config_dict.items():
        for sink in v["sinks"]:
            try:
                config_dict[sink]["sources"].append(k)
            except KeyError:
                added_entry = [sink, {"module_type": "sink", "state": "low", "sources": [k], "sinks": []}]

    # Set state for &-type modules
    for k, v in config_dict.items():
        if config_dict[k]["module_type"] == "&":
            for source in v["sources"]:
                config_dict[k]["module_state"].append(config_dict[source]["output"])
    if added_entry:
        config_dict[added_entry[0]] = added_entry[1]

    return config_dict

In [4]:
def process_signal(module_states: dict, module_name: str, input_signal: str, input_source: str) -> tuple[str, dict]:
    module_type = module_states[module_name]["module_type"]
    if module_type == "%":
        if input_signal == "high":
            return ("none", module_states)
        if module_states[module_name]["module_state"] == "off":
            module_states[module_name]["module_state"] = "on"
            module_states[module_name]["output"] = "high"
            return ("high", module_states)
        else:
            module_states[module_name]["module_state"] = "off"
            module_states[module_name]["output"] = "low"
            return ("low", module_states)

    elif module_type == "&":
        # update module state
        input_index = module_states[module_name]["sources"].index(input_source)
        module_states[module_name]["module_state"][input_index] = input_signal
        if all([value == "high" for value in module_states[module_name]["module_state"]]):
            return ("low", module_states)
        return ("high", module_states)
    else:
        return (input_signal, module_states)

def push_button(module_states: dict) -> dict:
    signal_sequence = deque()
    signal_sequence.append(("broadcaster", "low", "button"))

    while signal_sequence:
        signal_input_module, signal_value, signal_source_module = signal_sequence.popleft()
        SIGNAL_HISTORY.append((signal_source_module, signal_value, signal_input_module))
        output_signal, module_states = process_signal(module_states, signal_input_module, signal_value, signal_source_module)
        if output_signal != "none":
            for output_module in module_states[signal_input_module]["sinks"]:
                signal_sequence.append((output_module, output_signal, signal_input_module))
    return module_states

In [5]:
module_states = init_modules_inputs_outputs_state(module_config=data)
SIGNAL_HISTORY = []
for _ in range(1000):
    module_states = push_button(module_states=module_states)

print(len([x for x in SIGNAL_HISTORY if x[1] == "low"])*len([x for x in SIGNAL_HISTORY if x[1] == "high"]))

834323022


# Pt. 2

In [19]:
import math

module_states = init_modules_inputs_outputs_state(module_config=data)
SIGNAL_HISTORY = []
presses = 0

# Get the important signals, all rs inputs high triggers rx low
source_repetition = {x: 0 for x in module_states["rs"]["sources"]}

while True:
    presses += 1
    module_states = push_button(module_states=module_states)

    for source in source_repetition.keys():
        if (source, "high", "rs") in SIGNAL_HISTORY:
            source_repetition[source] = presses
    SIGNAL_HISTORY = []
    if all([value != 0 for value in source_repetition.values()]):
        break
    
math.lcm(*[value for value in source_repetition.values()])

225386464601017